ATOM      1  OH  OSP3    1       4.013   0.831  -9.083  1.00  0.00              
ATOM      2 1HH  OSP3    1       4.941   0.844  -8.837  1.00  0.00              
ATOM      3 2HH  OSP3    1       3.750  -0.068  -9.293  1.00  0.00       

In [1]:
coord=[[4.013,   0.831 , -9.083],[4.941,   0.844 , -8.837],[3.750,  -0.068,  -9.293]]

In [2]:
import numpy as np

In [3]:
coord=np.array(coord)

In [4]:
coord

array([[ 4.013,  0.831, -9.083],
       [ 4.941,  0.844, -8.837],
       [ 3.75 , -0.068, -9.293]])

In [5]:
coord=coord-coord.mean(axis=0)

In [6]:
coord

array([[-0.22166667,  0.29533333, -0.012     ],
       [ 0.70633333,  0.30833333,  0.234     ],
       [-0.48466667, -0.60366667, -0.222     ]])

In [93]:
np.sum((coord[0]-coord[1])**2)**0.5

0.9601400939446285

In [94]:
np.sum((coord[2]-coord[1])**2)**0.5

1.5678523527424384

In [95]:
np.sum((coord[0]-coord[2])**2)**0.5

0.9599322892787799

In [7]:
import k3d

In [8]:
pl=k3d.plot()
pl += k3d.points(np.array(coord,dtype='float32'))
pl.display()

Output()

In [10]:
from scipy.spatial.transform import Rotation as R


In [44]:
R.create_group?

In [111]:
R1 = R.create_group('C4',axis='Z')
R2 = R.create_group('C2',axis='X')

In [112]:
R.random().as_matrix()

array([[-0.2049241 , -0.21780283,  0.95423689],
       [ 0.78819475, -0.61474452,  0.02895193],
       [ 0.58030608,  0.75805746,  0.29764701]])

In [125]:
l=2.0
coordr = np.einsum('ij,jm->im',coord,R.random().as_matrix())
print(coordr)
coords = np.einsum('ix,jxy->jiy',coordr+np.array([l,l,l]),R1.as_matrix())
coords2 = np.einsum('ix,jxy->jiy',coordr+np.array([l,-l,-l]),R1.as_matrix())

[[ 0.13514469 -0.28177262 -0.19708372]
 [-0.47076193  0.3856909  -0.5275964 ]
 [ 0.33561724 -0.10391828  0.72468012]]


In [126]:
coords

array([[[ 2.13514469,  1.71822738,  1.80291628],
        [ 1.52923807,  2.3856909 ,  1.4724036 ],
        [ 2.33561724,  1.89608172,  2.72468012]],

       [[ 1.71822738, -2.13514469,  1.80291628],
        [ 2.3856909 , -1.52923807,  1.4724036 ],
        [ 1.89608172, -2.33561724,  2.72468012]],

       [[-2.13514469, -1.71822738,  1.80291628],
        [-1.52923807, -2.3856909 ,  1.4724036 ],
        [-2.33561724, -1.89608172,  2.72468012]],

       [[-1.71822738,  2.13514469,  1.80291628],
        [-2.3856909 ,  1.52923807,  1.4724036 ],
        [-1.89608172,  2.33561724,  2.72468012]]])

In [127]:
pl=k3d.plot()
pl += k3d.points(np.array(coords,dtype='float32'))
pl += k3d.points(np.array(coords2,dtype='float32'))
pl.display()

Output()

In [128]:
coordf=np.concatenate([coords,coords2])

In [129]:
coordf.shape

(8, 3, 3)

In [130]:
types=['O', 'H', 'H']
for imol in range(coordf.shape[0]):
    for itype in range(coordf.shape[1]):
        x,y,z=coordf[imol][itype]/0.529
        print(f'{types[itype]} {x} {y} {z}')

O 4.036190336140067 3.2480668862142097 3.4081593110550488
H 2.890809201509836 4.509812656771914 2.783371649786892
H 4.415155471801892 3.5842754664656713 5.150624048609854
O 3.2480668862142106 -4.036190336140066 3.4081593110550488
H 4.509812656771915 -2.890809201509835 2.783371649786892
H 3.584275466465672 -4.415155471801891 5.150624048609854
O -4.036190336140067 -3.24806688621421 3.4081593110550488
H -2.8908092015098354 -4.509812656771914 2.783371649786892
H -4.415155471801891 -3.5842754664656717 5.150624048609854
O -3.2480668862142106 4.036190336140066 3.4081593110550488
H -4.509812656771915 2.890809201509835 2.783371649786892
H -3.584275466465672 4.415155471801891 5.150624048609854
O 4.036190336140067 -4.313369786753654 -4.1532773619128145
H 2.890809201509836 -3.05162401619595 -4.778065023180971
H 4.415155471801892 -3.9771612065021924 -2.4108126243580097
O -4.313369786753653 -4.036190336140068 -4.1532773619128145
H -3.051624016195949 -2.890809201509837 -4.778065023180971
H -3.9771612